In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# BigFrames Multimodal DataFrame

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/googleapis/python-bigquery-dataframes/blob/main/notebooks/multimodal/multimodal_dataframe.ipynb">
      <img src="../../third_party/logo/colab-logo.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/multimodal/multimodal_dataframe.ipynb">
      <img src="../../third_party/logo/github-logo.png" width="32" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/multimodal/multimodal_dataframe.ipynb">
      <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTW1gvOovVlbZAIZylUtf5Iu8-693qS1w5NJw&s" alt="BQ logo" width="35">
      Open in BQ Studio
    </a>
  </td>
</table>


This notebook is introducing BigFrames Multimodal features:
1. Create Multimodal DataFrame
2. Combine unstructured data with structured data
3. Conduct image transformations
4. Use LLM models to ask questions and generate embeddings on images
5. PDF chunking function

### Setup

Install the latest bigframes package if bigframes version < 2.4.0

In [ ]:
# !pip install bigframes --upgrade

In [ ]:
PROJECT = "bigframes-dev" # replace with your project. 
# Refer to https://cloud.google.com/bigquery/docs/multimodal-data-dataframes-tutorial#required_roles for your required permissions

OUTPUT_BUCKET = "bigframes_blob_test" # replace with your GCS bucket. 
# The connection (or bigframes-default-connection of the project) must have read/write permission to the bucket. 
# Refer to https://cloud.google.com/bigquery/docs/multimodal-data-dataframes-tutorial#grant-permissions for setting up connection service account permissions.
# In this Notebook it uses bigframes-default-connection by default. You can also bring in your own connections in each method.

import bigframes
# Setup project
bigframes.options.bigquery.project = PROJECT

# Display options
bigframes.options.display.blob_display_width = 300
bigframes.options.display.progress_bar = None

import bigframes.pandas as bpd

### 1. Create Multimodal DataFrame
There are several ways to create Multimodal DataFrame. The easiest way is from the wildcard paths.

In [ ]:
# Create blob columns from wildcard path.
df_image = bpd.from_glob_path(
    "gs://cloud-samples-data/bigquery/tutorials/cymbal-pets/images/*", name="image"
)
# Other ways are: from string uri column
# df = bpd.DataFrame({"uri": ["gs://<my_bucket>/<my_file_0>", "gs://<my_bucket>/<my_file_1>"]})
# df["blob_col"] = df["uri"].str.to_blob()

# From an existing object table
# df = bpd.read_gbq_object_table("<my_object_table>", name="blob_col")

In [ ]:
# Take only the 5 images to deal with. Preview the content of the Mutimodal DataFrame
df_image = df_image.head(5)
df_image

### 2. Combine unstructured data with structured data

Now you can put more information into the table to describe the files. Such as author info from inputs, or other metadata from the gcs object itself.

In [ ]:
# Combine unstructured data with structured data
df_image["author"] = ["alice", "bob", "bob", "alice", "bob"]  # type: ignore
df_image["content_type"] = df_image["image"].blob.content_type()
df_image["size"] = df_image["image"].blob.size()
df_image["updated"] = df_image["image"].blob.updated()
df_image

Then you can filter the rows based on the structured data. And for different content types, you can display them respectively or together.

In [ ]:
# filter images and display, you can also display audio and video types
df_image[df_image["author"] == "alice"]["image"].blob.display()

### 3. Conduct image transformations
BigFrames Multimodal DataFrame provides image(and other) transformation functions. Such as image_blur, image_resize and image_normalize. The output can be saved to GCS folders or to BQ as bytes.

In [ ]:
df_image["blurred"] = df_image["image"].blob.image_blur(
    (20, 20), dst=f"gs://{OUTPUT_BUCKET}/image_blur_transformed/", engine="opencv"
)
df_image["resized"] = df_image["image"].blob.image_resize(
    (300, 200), dst=f"gs://{OUTPUT_BUCKET}/image_resize_transformed/", engine="opencv"
)
df_image["normalized"] = df_image["image"].blob.image_normalize(
    alpha=50.0,
    beta=150.0,
    norm_type="minmax",
    dst=f"gs://{OUTPUT_BUCKET}/image_normalize_transformed/",
    engine="opencv",
)

In [ ]:
# You can also chain functions together
df_image["blur_resized"] = df_image["blurred"].blob.image_resize((300, 200), dst=f"gs://{OUTPUT_BUCKET}/image_blur_resize_transformed/", engine="opencv")

In [ ]:
df_image

### 4. Use LLM models to ask questions and generate embeddings on images

In [ ]:
from bigframes.ml import llm
gemini = llm.GeminiTextGenerator()

In [ ]:
# Ask the same question on the images
df_image = df_image.head(2)
answer = gemini.predict(df_image, prompt=["what item is it?", df_image["image"]])
answer[["ml_generate_text_llm_result", "image"]]

In [ ]:
# Ask different questions
df_image["question"] = ["what item is it?", "what color is the picture?"]

In [ ]:
answer_alt = gemini.predict(df_image, prompt=[df_image["question"], df_image["image"]])
answer_alt[["ml_generate_text_llm_result", "image"]]

In [ ]:
# Generate embeddings.
embed_model = llm.MultimodalEmbeddingGenerator()
embeddings = embed_model.predict(df_image["image"])
embeddings

### 5. PDF chunking function

In [ ]:
df_pdf = bpd.from_glob_path("gs://cloud-samples-data/bigquery/tutorials/cymbal-pets/documents/*", name="pdf")

In [ ]:
df_pdf["chunked"] = df_pdf["pdf"].blob.pdf_chunk(engine="pypdf")

In [ ]:
chunked = df_pdf["chunked"].explode()
chunked